In [2]:
%matplotlib inline
import pandas as pd
import urllib2
from pyproj import Proj, transform
import xmltodict
import numpy as np
from datetime import datetime, date, timedelta
import mechanize
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

#Functions

In [3]:
def getelev(x):
    '''
    Input
    x[0] = UTM X
    x[1] = UTM Y
    
    Output
    Elevation
    '''
    elev = "http://ned.usgs.gov/epqs/pqs.php?x="+str(x[0])+"&y="+str(x[1])+"&units=Meters&output=xml"
    response = urllib2.urlopen(elev)
    html = response.read()
    d = xmltodict.parse(html)
    return float(d['USGS_Elevation_Point_Query_Service']['Elevation_Query']['Elevation'])

In [4]:
def getwlelev(x):
    return x[1] - (x[0]/3.2808)

In [5]:
def qqq(x):
    x.rstrip().lstrip()
    j = x.split(' ')
    a = j[0][:1]
    b = j[0][1:]
    c = j[1][:1]
    d = j[1][1:]
    e = [a,b,c,d,j[2],j[3],j[4],j[5],j[6]]
    
    NS = int(e[1].replace(',',''))
    EW = int(e[3].replace(',',''))
    qc = e[4]
    d1 = e[0]
    d2 = e[2]
    dic1 = {'NE':'a','NW':'b','SW':'c','SE':'d'}
    qcdDict = {'E4S':'d','E4N':'a','N4E':'a','N4W':'b','W4N':'b','W4S':'c','S4W':'c','S4E':'d'}
    dic2 = {'a':'b','b':'a','c':'d','d':'c'}
    dic3 = {'a':'d','b':'c','c':'b','d':'a'}
    dic4 = {'a':'c','b':'d','c':'a','d':'b'}
    if qc[-1]=='4':
        if qc[0]=='N' or qc[0]=='S':
            qcd = qc+d2
        elif qc[0]=='E' or qc[0]=='W':
            qcd = qc+d1
        q1 = qcdDict.get(qcd,'x')
    elif qc in ('NE','NW','SW','SE'):
        q1 = dic1.get(qc)
    else:
        print "invalid quarter"
        q1 = 'X'
    if NS < 1320:
        if EW <1320:
            q2 = q1
        elif EW >1320:
            qd2 = {'a':'b','b':'a','c':'d','d':'c'}
            q2 = dic2.get(q1,'x')
    elif NS > 1320:
        if EW <1320:
            q2 = dic3.get(q1,'x')
        elif EW >1320:
            q2 = dic4.get(q1,'x')
    else:
        q2 = 'X'

    if NS < 660 or (NS > 1320 and NS < 1980):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = q1
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic2.get(q1,'x')
    elif (NS > 660 and NS < 1320) or (NS > 1980 and NS < 2640):
        if (EW < 660) or (EW > 1320 and EW < 1980):
            q3 = dic3.get(q1,'x')
        elif (EW > 660 and EW < 1320) or (EW > 1980 and EW < 2640):
            q3 = dic4.get(q1,'x')
    else:
        q3 = 'X'
    Tn = e[6][:-1].rjust(2)
    Rn = e[7][:-1].rjust(2)
    Sec = e[5].rjust(2)
    TRd = e[6][-1]+e[7][-1]
    TR = dic1.get(TRd).upper()
    CAD = '('+TR+'-'+Tn+'-'+Rn+')'+Sec+q1+q2+q3+'-1'
    return CAD 

In [6]:
def proj(x):
    inProj = Proj(init='epsg:4326') #WGS84
    outProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2, y2

def projy(x):
    inProj = Proj(init='epsg:4326') #WGS84
    outProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return y2

def projx(x):
    inProj = Proj(init='epsg:4326') #WGS84
    outProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2

In [7]:
def revproj(x):
    outProj = Proj(init='epsg:4326') #WGS84
    inProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2, y2

def revprojy(x):
    outProj = Proj(init='epsg:4326') #WGS84
    inProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return y2

def revprojx(x):
    outProj = Proj(init='epsg:4326') #WGS84
    inProj = Proj(init='epsg:2152') #NAD83(CSRS98) / UTM zone 12N
    x2,y2 = transform(inProj,outProj,x[0],x[1])
    return x2

In [8]:
def getwellinfo(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS, CAD

def getwellPLSS(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS

def getwellCAD(x):
    request = mechanize.Request("http://maps.waterrights.utah.gov/asp/location.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0]
    form["UTMx"]= str(x[0])
    form["UTMy"]= str(x[1])
    form["datumutm"]=["NAD83"]
    desc =  mechanize.urlopen(form.click()).read()
    try:
        PLSS, CAD = getPLSS(desc)
    except(ValueError):
        PLSS, CAD = np.nan, np.nan
    return PLSS

In [9]:
def winmatch(x):
    request = mechanize.Request("http://waterrights.utah.gov/wellinfo/wellsearch.asp")
    response = mechanize.urlopen(request)
    forms = mechanize.ParseResponse(response, backwards_compat=False)
    response.close()
    form = forms[0] 
    #print form
    form["mainoption"]=["radius"]
    form["SearchRadius"]="2000"
    form["option"]=["UTM"]
    form["xUTM"]=str(x[0])
    form["yUTM"]=str(x[1])

    win =  mechanize.urlopen(form.click()).read()
    winbeg = win.find('WIN=')
    if winbeg == -1:
        return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan 
    else:
        wintabeg=win.find('<table',win.find('<table')+5)
        wintaend=win.find('</table>')
        winmatches = pd.read_html(win[wintabeg:wintaend], header=0, skiprows=0)
        winmatches = winmatches[0]
        winDic = {u'WRNUM/Appl. No.':'WRNUM',u'Distance From Point (ft)':'DIST',u'Diameter':'Diam',u'Depth':'TD',
          u'Drilled Date':'DrillDate',u'Location(link to Log)':'Locatio',u'WIN':'WIN',u'Geologic Log':'Log'}
        winmatches.rename(columns=winDic,inplace=True)
        return winmatches.ix[0,0], winmatches.ix[0,1], winmatches.ix[0,2], winmatches.ix[0,3], winmatches.ix[0,4], winmatches.ix[0,5], int(winmatches.ix[0,6])

In [10]:
def unitassign(x):
    clay = x[0]
    silt = x[1]
    sand = x[2]
    gravel = x[3]
    cobbles = x[4]
    boulders = x[5]
    hardpan = x[6]
    conglomerate = x[7]
    bedrock = x[8]
    other = x[9]
    unitlist = [clay,silt,sand,gravel,cobbles,boulders, hardpan,conglomerate,bedrock,other]
    unitindex = ['clay','silt','sand','gravel','cobbles','boulders', 'hardpan','conglomerate','bedrock','other']
    unitsum = np.sum(unitlist)
    j =str("")
    for i in range(len(unitlist)):
        if unitlist[i] == 1:
            if len(j)==0:
                j = unitindex[i]
            else:
                j = j + "-" + unitindex[i]
    return j    
        
        

#Analysis

In [11]:
logroot = "E:\\PROJECTS\\UMAR\\Data\\WELL_LOGS\\"
logfile = logroot + "well_logs_v2.xlsx"

In [12]:
local = pd.read_excel(logfile,"wells_in_AOR")

In [13]:
lith = pd.read_excel(logfile,"Appendix")
casing = pd.read_excel(logfile,"casing")
screen = pd.read_excel(logfile,"Screen")

In [14]:
local['lonX'] = local[['POINT_X','POINT_Y']].apply(lambda x: revprojx(x),1)
local['latY'] =local[['POINT_X','POINT_Y']].apply(lambda x: revprojy(x),1)

In [16]:
local['elev_m'] = local[['lonX','latY']].apply(lambda x: getelev(x),1)

In [17]:
local['WRNUM2'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[0],1)
local['Dist'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[1],1)
local['Diameter'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[2],1)
local['DepthWR'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[3],1)
local['DrillDate'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[4],1)
local['Loc']= local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[5],1)
local['WIN2'] = local[['POINT_X','POINT_Y']].apply(lambda x: winmatch(x)[6],1)

In [18]:
lith = pd.merge(lith, local, on='WIN', how='left')
lith['from_elevm'] = lith[['ftFROM','elev_m']].apply(lambda x: getwlelev(x),1)
lith['to_elevm'] = lith[['ftto','elev_m']].apply(lambda x: getwlelev(x),1)
lith.drop([u'TYPE', u'PRIORITY', u'USES', u'CFS', u'ACFT', u'Dist', u'DrillDate', u'Loc', u'ftNS',u'ftEW', u'Quad', 
           u'Section', u'T', u'R', u'OWNER', u'SOURCE', u'Link'], inplace=True, axis=1)

In [19]:
screen = pd.merge(screen, local, on='WIN', how='left')
screen['from_elevm'] = screen[['ftFROM','elev_m']].apply(lambda x: getwlelev(x),1)
screen['to_elevm'] = screen[['ftto','elev_m']].apply(lambda x: getwlelev(x),1)
screen.drop([u'TYPE', u'PRIORITY', u'USES', u'CFS', u'ACFT', u'Dist', u'DrillDate', u'Loc', u'ftNS',u'ftEW', u'Quad', 
           u'Section', u'T', u'R', u'OWNER', u'SOURCE', u'Link'], inplace=True, axis=1)

In [32]:
screen['from_m'] = screen['ftFROM'].apply(lambda x: x/3.2808,1)
screen['to_m'] = screen['ftto'].apply(lambda x: x/3.2808,1)

In [20]:
casing = pd.merge(casing, local, on='WIN', how='left')
casing['from_elevm'] = casing[['ftFROM','elev_m']].apply(lambda x: getwlelev(x),1)
casing['to_elevm'] = casing[['ftto','elev_m']].apply(lambda x: getwlelev(x),1)
casing.drop([u'TYPE', u'PRIORITY', u'USES', u'CFS', u'ACFT', u'Dist', u'DrillDate', u'Loc', u'ftNS',u'ftEW', u'Quad', 
           u'Section', u'T', u'R', u'OWNER', u'SOURCE', u'Link'], inplace=True, axis=1)

In [21]:
lith.fillna(0, inplace=True)

In [22]:
lith.units = lith[['clay','silt','sand','gravel','cobbles','boulders', 'hardpan','conglomerate','bedrock','other']].apply(lambda x: unitassign(x),1)

In [23]:
consdict = {'other':'other', 'boulders':'gravel', 'gravel':'gravel', 'sand-gravel-cobbles':'sand-gravel',
            'sand-gravel-cobbles-boulders':'sand-gravel', 'clay-boulders':'clay-gravel', 
            'clay-gravel-boulders':'clay-gravel', 'gravel-conglomerate':'conglomerate', 'cobbles':'gravel',
            'gravel-cobbles':'gravel', 'gravel-boulders':'gravel', 'clay-gravel-cobbles-boulders':'clay-gravel', 
            'gravel-cobbles-boulders':'gravel', 'clay-cobbles-boulders':'clay-gravel', 
            'clay-cobbles':'clay-gravel','clay-sand-gravel-cobbles':'clay-gravel', 
            'clay-hardpan':'hardpan', 'cobbles-boulders':'gravel', 'clay-gravel-cobbles':'clay-gravel', 
            'clay-conglomerate':'conglomerate', 'clay-silt-sand-gravel-conglomerate':'conglomerate', 
            'sand-gravel-boulders':'sand-gravel','sand-boulders':'sand-gravel','clay-silt-gravel':'clay-gravel',
           'clay-silt-sand':'clay-sand','clay-silt':'clay-sand','clay-sand-gravel':'clay-gravel',
           'silt-sand':'sand','clay-silt-gravel-cobbles':'clay-gravel','silt-sand-gravel':'sand-gravel'}
lith['unitssimp'] = lith.units.apply(lambda x:consdict.get(x,x),1)

In [24]:
lith.units.unique()

array(['other', 'sand-gravel', 'clay', '', 'gravel', 'sand', 'boulders',
       'clay-silt-gravel', 'clay-silt-sand', 'clay-sand-gravel',
       'clay-sand', 'clay-gravel', 'sand-gravel-cobbles',
       'sand-gravel-cobbles-boulders', 'clay-boulders', 'conglomerate',
       'clay-gravel-boulders', 'gravel-conglomerate', 'cobbles',
       'gravel-cobbles', 'silt-sand-gravel', 'hardpan', 'gravel-boulders',
       'bedrock', 'clay-silt', 'clay-silt-gravel-cobbles',
       'clay-gravel-cobbles-boulders', 'gravel-cobbles-boulders',
       'sand-gravel-other', 'clay-cobbles-boulders', 'clay-cobbles',
       'clay-silt-sand-gravel', 'clay-sand-gravel-cobbles', 'silt-sand',
       'clay-hardpan', 'clay-silt-other', 'silt', 'cobbles-boulders',
       'clay-gravel-cobbles', 'clay-other', 'clay-conglomerate',
       'clay-silt-sand-gravel-conglomerate', 'sand-gravel-boulders',
       'sand-boulders'], dtype=object)

In [25]:
lith['unitssimp'].unique()

array(['other', 'sand-gravel', 'clay', '', 'gravel', 'sand', 'clay-gravel',
       'clay-sand', 'conglomerate', 'hardpan', 'bedrock',
       'sand-gravel-other', 'clay-silt-sand-gravel', 'clay-silt-other',
       'silt', 'clay-other'], dtype=object)

In [26]:
def otherassign(x):
    if x[0] == 'other' or x[0]=='':
        if str(x[1]).lower().find('soil') >-1:
            return 'soil'
        elif str(x[1]).lower().find('overburden') >-1:
            return 'soil'
        elif str(x[1]).lower().find('limestone') >-1:
            return 'limestone'
        elif str(x[1]).lower().find('shale') >-1:
            return 'shale'
        elif str(x[1]).lower().find('cemented') >-1:
            return 'conglomerate'
        elif str(x[1]).lower().find('conglomerate') >-1:
            return 'conglomerate'
        else:
            return ''
    elif x[0] == 'bedrock':
        if str(x[1]).lower().find('limestone') >-1:
            return 'limestone'
        elif str(x[1]).lower().find('shale') >-1:
            return 'shale'
        elif str(x[1]).lower().find('cemented') >-1:
            return 'conglomerate'
        elif str(x[1]).lower().find('conglomerate') >-1:
            return 'conglomerate'
        else:
            return 'bedrock'
    else:
        return x[0]
        

In [27]:
lith['unitssimp'] = lith[['unitssimp','comments']].apply(lambda x:otherassign(x),1)

In [28]:
lith['unitssimp'].unique()

array(['soil', 'sand-gravel', 'clay', '', 'gravel', 'sand', 'clay-gravel',
       'clay-sand', 'conglomerate', 'hardpan', 'bedrock', 'limestone',
       'sand-gravel-other', 'clay-silt-sand-gravel', 'clay-silt-other',
       'silt', 'clay-other', 'shale'], dtype=object)

In [29]:
unitnumber = {'soil':0, 'sand-gravel':1, 'clay':2, 'gravel':1, 'sand':4, 'clay-gravel':5,
              'clay-sand':3, 'conglomerate':7, 'hardpan':2, 'bedrock':6, 'limestone':6,
              'sand-gravel-other':1, 'clay-silt-sand-gravel':3, 'clay-silt-other':2,
              'silt':2, 'clay-other':2, 'shale':2}
lith['unitnumber'] = lith['unitssimp'].apply(lambda x: unitnumber.get(x,8),1)

<tbody><tr id="boreholelog_table">
<td colspan="3" style="background:#4545AA; color:White; text-align:left;"><a href="http://www.archydrogw.com/Arc_Hydro_Groundwater_Data_Model" class="image"><img alt="GIS Table.png" src="http://www.archydrogw.com/images/thumb/6/65/GIS_Table.png/20px-GIS_Table.png" width="20" height="15" srcset="/images/6/65/GIS_Table.png 1.5x, /images/6/65/GIS_Table.png 2x"></a> <b>BoreholeLog</b>
</td></tr>
<tr>
<td colspan="3"> Table
</td></tr>
<tr style="background:LightGray;">
<td style="width: 100px"> <b>Field name</b>
</td>
<td style="width: 75px"> <b>Type</b>
</td>
<td> <b>Description</b>
</td></tr>
<tr style="background:WhiteSmoke;">
<td> WellID </td>
<td> Long Integer </td>
<td> References the HydroID of a Well feature.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> RefElev </td>
<td> Double </td>
<td> A reference elevation (e.g., land elevation, top of casing) giving the starting elevations for data measured as depth along the borehole.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> FromDepth </td>
<td> Double </td>
<td> The top elevation of an interval measured as depth along the borehole.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> ToDepth </td>
<td> Double </td>
<td> The bottom elevation of an interval measured as depth along the borehole.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> TopElev </td>
<td> Double </td>
<td> Top elevation of an interval  represented in absolute elevation units (e.g., feet above mean sea level).
</td></tr>
<tr style="background:WhiteSmoke;">
<td> BottomElev </td>
<td> Double </td>
<td> Bottom elevation of an interval  represented in absolute elevation units (e.g., feet above mean sea level).
</td></tr>
<tr style="background:WhiteSmoke;">
<td> ElevUnits </td>
<td> Text </td>
<td> Units of elevations stored in the TopElev and BottomElev attributes.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> Material </td>
<td> Text </td>
<td> Description of strata observed along a borehole. Usually documented in drilling logs and later classified into geologic/hydrogeologic units.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUID </td>
<td> Long Integer </td>
<td> Hydrogeologic unit identifier. Classifies borehole data into hydrogeologic units defined in the HydrogeologicUnit table.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUCode </td>
<td> Text </td>
<td> Hydrogeologic unit code. Text for classifying, symbolizing,  and labeling hydrogeologic units.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> LogType </td>
<td> Text </td>
<td> Distinguishes between types of borehole logs (e.g., well completion, hydrostratigraphy).
</td></tr></tbody>

In [30]:
boreholematch = {'WIN':'WellID', 'elev_m':'RefElev', 'ftFROM':'FromDepth', 'ftto':'ToDepth',
                 'from_elevm':'TopElev', 'to_elevm':'BottomElev', 'unitssimp':'Material', 
                 'unitnumber':'HGUID'}
BoreholeLog = lith.rename(columns=boreholematch)

In [31]:
BoreholeLog

,WellID,FromDepth,ToDepth,ftthickness,water,highperm,lowperm,clay,silt,sand,...,latY,RefElev,WRNUM2,Diameter,DepthWR,WIN2,TopElev,BottomElev,Material,HGUID
0,435,0,6,6,0,0,0,0,0,0,...,41.696700,1369.715698,25-9157,6,89,435,1369.715698,1367.886876,soil,0
1,435,6,13,7,0,1,0,0,0,1,...,41.696700,1369.715698,25-9157,6,89,435,1367.886876,1365.753250,sand-gravel,1
2,435,13,19,6,0,0,1,1,0,0,...,41.696700,1369.715698,25-9157,6,89,435,1365.753250,1363.924428,clay,2
3,435,19,49,30,0,0,0,0,0,0,...,41.696700,1369.715698,25-9157,6,89,435,1363.924428,1354.780316,,8
4,435,49,65,16,0,0,1,1,0,0,...,41.696700,1369.715698,25-9157,6,89,435,1354.780316,1349.903457,clay,2
5,435,65,75,10,0,1,0,0,0,1,...,41.696700,1369.715698,25-9157,6,89,435,1349.903457,1346.855420,sand-gravel,1
6,435,75,90,15,0,1,0,0,0,0,...,41.696700,1369.715698,25-9157,6,89,435,1346.855420,1342.283364,gravel,1
7,435,90,100,10,0,1,0,0,0,1,...,41.696700,1369.715698,25-9157,6,89,435,1342.283364,1339.235327,sand,4
8,1581,0,3,3,0,0,0,0,0,0,...,41.674291,1382.720459,25-8768,6,79,1581,1382.720459,1381.806048,soil,0
9,1581,3,22,19,0,1,0,0,0,0,...,41.674291,1382.720459,25-8768,6,79,1581,1381.806048,1376.014777,gravel,1


<tbody><tr id="borepoint_table">
<td colspan="3" style="background:#4545AA; color:White; text-align:left;"><a href="http://www.archydrogw.com/Arc_Hydro_Groundwater_Data_Model" class="image"><img alt="GIS Point.png" src="http://www.archydrogw.com/images/thumb/8/84/GIS_Point.png/20px-GIS_Point.png" width="20" height="16" srcset="/images/thumb/8/84/GIS_Point.png/30px-GIS_Point.png 1.5x, /images/8/84/GIS_Point.png 2x"></a> <b>BorePoint</b>
</td></tr>
<tr>
<td colspan="3"> Point Z Feature Class
</td></tr>
<tr style="background:LightGray;">
<td style="width: 100px"> <b>Field name</b>
</td>
<td style="width: 75px"> <b>Type</b>
</td>
<td> <b>Description</b>
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HydroID </td>
<td> Long Integer </td>
<td> Unique feature identifier in the geodatabase used for creating relationships between classes of the data model.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HydroCode </td>
<td> Text </td>
<td> Permanent public identifier of the feature used for relating features with external information systems.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> WellID </td>
<td> Long Integer </td>
<td> References the HydroID of a Well feature.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> GeoSectionID </td>
<td> Long Integer </td>
<td>
</td></tr>
<tr style="background:WhiteSmoke;">
<td> Material </td>
<td> Text </td>
<td> Description of strata observed along a borehole. Usually documented in drilling logs and later classified into geologic/hydrogeologic units.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUID </td>
<td> Long Integer </td>
<td> Hydrogeologic unit identifier. Classifies borehole data into hydrogeologic units defined in the HydrogeologicUnit table.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUCode </td>
<td> Text </td>
<td> Hydrogeologic unit code. Text for classifying, symbolizing,  and labeling hydrogeologic units.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> PointElev </td>
<td> Double </td>
<td> Elevation of the BorePoint feature.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> FType </td>
<td> Text </td>
<td> Distinguishes between types of BorePoint features (e.g., stratigraphic picks, sampling ports).
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HorizonID </td>
<td> Long Integer </td>
<td>
</td></tr></tbody>

In [33]:
local.columns

Index([    u'WRNUM',       u'WIN',     u'Depth',      u'Diam',      u'TYPE',
        u'PRIORITY',      u'USES',       u'CFS',      u'ACFT',      u'ftNS',
            u'ftEW',      u'Quad',   u'Section',         u'T',         u'R',
           u'OWNER',    u'SOURCE',       u'DWS',   u'Sampled',      u'Link',
         u'POINT_X',   u'POINT_Y',      u'lonX',      u'latY',    u'elev_m',
          u'WRNUM2',      u'Dist',  u'Diameter',   u'DepthWR', u'DrillDate',
             u'Loc',      u'WIN2'],
      dtype='object')

<tbody><tr id="well_table">
<td colspan="3" style="background:#4545AA; color:White; text-align:left;"><a href="http://www.archydrogw.com/Arc_Hydro_Groundwater_Data_Model" class="image"><img alt="GIS Point.png" src="http://www.archydrogw.com/images/thumb/8/84/GIS_Point.png/20px-GIS_Point.png" width="20" height="16" srcset="/images/thumb/8/84/GIS_Point.png/30px-GIS_Point.png 1.5x, /images/8/84/GIS_Point.png 2x"></a> <b>Well</b>
</td></tr>
<tr>
<td colspan="3"> Point Feature Class
</td></tr>
<tr style="background:LightGray;">
<td style="width: 100px"> <b>Field name</b>
</td>
<td style="width: 75px"> <b>Type</b>
</td>
<td> <b>Description</b>
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HydroID </td>
<td> Long Integer </td>
<td> Unique feature identifier in the geodatabase used for creating relationships between classes of the data model.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HydroCode </td>
<td> Text </td>
<td> Permanent public identifier of the feature used for relating features with external information systems.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> LandElev </td>
<td> Double </td>
<td> The elevation of the land surface at the well location. Is commonly used to reference vertical information (measured as depth along the well).
</td></tr>
<tr style="background:WhiteSmoke;">
<td> WellDepth </td>
<td> Double </td>
<td> The depth of the well. Together with the LandElev provides a description of the well’s 3D geometry.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> AquiferID </td>
<td> Long Integer </td>
<td> Relates a Well feature with an Aquifer feature. The AquiferID of a Well feature is equal to the HydroID of an Aquifer feature.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> AqCode </td>
<td> Text </td>
<td> Text describing the aquifer. Is used to symbolize wells based on the related aquifer.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUID </td>
<td> Long Integer </td>
<td> Relates the well to a hydrogeologic unit.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> FType </td>
<td> Text </td>
<td> Distinguishes between types of wells (e.g., domestic, water supply, industrial).
</td></tr></tbody>

In [34]:
welllocal = {'WIN':'HydroID', 'elev_m':'LandElev','POINT_X':'X','POINT_Y':'Y'}

Well = local.rename(columns=welllocal)

<tbody><tr id="hydrogeologicunit_table">
<td colspan="3" style="background:#4545AA; color:White; text-align:left;"><a href="http://www.archydrogw.com/Arc_Hydro_Groundwater_Data_Model" class="image"><img alt="GIS Table.png" src="http://www.archydrogw.com/images/thumb/6/65/GIS_Table.png/20px-GIS_Table.png" width="20" height="15" srcset="/images/6/65/GIS_Table.png 1.5x, /images/6/65/GIS_Table.png 2x"></a> <b>HydrogeologicUnit</b>
</td></tr>
<tr>
<td colspan="3"> Table
</td></tr>
<tr style="background:LightGray;">
<td style="width: 100px"> <b>Field name</b>
</td>
<td style="width: 75px"> <b>Type</b>
</td>
<td> <b>Description</b>
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HydroID </td>
<td> Long Integer </td>
<td> Unique identifier in the geodatabase used for creating relationships between classes of the data model. Used to relate between hydrogeologic units defined in the table and spatial features within the hydrostratigraphy component.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUCode </td>
<td> Text </td>
<td> Hydrogeologic unit code. The permanent identification code of hydrogeologic units, used to establish a linkage with external information systems.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HGUName </td>
<td> Text </td>
<td> Text descriptor of hydrogeologic units used for labeling and symbolization.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> AquiferID </td>
<td> Long Integer </td>
<td> Aquifer identifier for grouping hydrogeologic units. AquiferID is also used to relate hydrogeologic units in the table with Aquifer features.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> AqCode </td>
<td> Text </td>
<td> Aquifer code. Text descriptor of the aquifer used for labeling, symbolization, and querying.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> Description </td>
<td> Text </td>
<td> Text  for storing  detailed descriptions of hydrogeologic units.
</td></tr>
<tr style="background:WhiteSmoke;">
<td> HorizonID </td>
<td> Long Integer </td>
<td> Index for describing the depositional sequence of hydrogeologic units.
</td></tr></tbody>

In [35]:
HydroGeoUnit = pd.DataFrame.from_dict(unitnumber, orient='index')
HydroGeoUnit.reset_index(inplace=True)
HydroGeoUnit.columns = ['HGUName','HydroID']

In [36]:
Well = Well[Well.HydroID > 0 ]

In [37]:
BoreholeLog.to_csv(logroot+"BoreholeLog.csv", index=False)
HydroGeoUnit.to_csv(logroot+"HydroUnits.csv", index=False)
Well.to_csv(logroot+"Wells.csv", index=False)
screen.to_csv(logroot+"Screens.csv",index=False)

In [117]:
lith[lith['unitssimp']=='other']['comments'].unique()

array([], dtype=object)

In [118]:
writer = pd.ExcelWriter(logroot+'AOR_well_attributes.xlsx')
local.to_excel(writer,'local', index=False)
casing.to_excel(writer,'casing', index=False)
screen.to_excel(writer,'screen', index=False)
lith.to_excel(writer,'lith', index=False)
writer.save()

In [ ]:
lith.groupby(['WIN','unitssimp'])['ftthickness'].sum()